### Functions for stepFunction File

In [ ]:
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt

def mask_raster_by_geometries(shapefile_path, tiff_path, output_dir, year, visualize=False):
    """
    Mask a raster by each geometry in a shapefile and save the output rasters using the 'unitcode' column as filenames.

    Parameters:
    - shapefile_path: str, path to the shapefile (.shp)
    - tiff_path: str, path to the TIFF file (.tif)
    - output_dir: str, directory to save the output masked raster files
    - visualize: bool, whether to visualize each masked raster (default: False)
    """

    # Load the shapefile
    gdf = gpd.read_file(shapefile_path)
    raster_basename = os.path.splitext(os.path.basename(tiff_path))[0]
    # Open the raster file
    with rasterio.open(tiff_path) as src:
        # Iterate through each geometry in the GeoDataFrame
        for index, row in gdf.iterrows():
            geom = row.geometry
            unitcode = int(row['unit_code'])

            # Mask the raster with the current geometry
            out_image, out_transform = mask(src, [geom], crop=True)

            # Copy and update the metadata
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

            # Define the output path using 'unitcode
            
            output_tiff_path = "{}/{}/{}/{}.tiff".format(output_dir, year, 
                                                    raster_basename, unitcode)
            
            # Create the output directory if it doesn't exist
            output_directory = os.path.dirname(output_tiff_path)
            if not os.path.exists(output_directory):
              os.makedirs(output_directory)

            # Save the masked raster to a new file
            with rasterio.open(output_tiff_path, "w", **out_meta) as dest:
                dest.write(out_image)

            # Optionally visualize the result for each mask
            if visualize:
                plt.imshow(out_image[0], cmap="gray")
                plt.title(f"Masked Raster for {unitcode}")
                plt.colorbar()
                plt.show()


In [ ]:
# Create Functions
def growingMonths(ranked_df_ir_rank):
  """
  Create a list of the growing months
  """
    
  def createList(r1, r2):
    # Create a list of the months betweeen planting and maturity
    return np.arange(r1, r2+1)

  def getListcom(p, m):
    # check if planting and maturity are in the same year
    if np.any(p < m):
      return createList(p, m)
    else:
      return np.concatenate((createList(p, 12), createList(1, m)))
        
  # Get the growing months 
  p = ranked_df_ir_rank.Planting_Month.tolist()[0]
  m = ranked_df_ir_rank.Maturity_Month.tolist()[0]
  
  return getListcom(p, m)

def rank_crops(df_area):
#     print(df_area)
    df_area = df_area.sort_values('Growing_area', ascending=False)
    df_area['rank'] = range(1, len(df_area) + 1)
#     print(df_area)
    return df_area.set_index('rank')

def custom_rank(sub_df):
  """
  Ranking function for a single state (unit_code)
  """
  
  # Define the type_order
  type_order = {'Perennial': 1, 'Other Perennial': 2, 'Fodder': 3, 'Annual': 4, 'Other Annual': 5}

  sub_df = sub_df.copy()  # Create a copy to avoid SettingWithCopyWarning
    
  # Redirect standard error to suppress warnings
  sys.stderr = open('nul', 'w')
    
  sub_df['Type_Rank'] = sub_df['Type'].map(type_order)
  sub_df = sub_df.sort_values(by=['Type_Rank', 'Growing_area'], ascending=[True, False])
  sub_df['rank'] = range(1, len(sub_df) + 1)
    
  # Reset standard error
  sys.stderr = sys.__stderr__
    
  return sub_df

def rankbyarea(CropCal_dir = "../data/CropCalender", irrigated_rainfed = "ir", year="00",
               by_state = False, options=('ir', 'rf')):
  """
  Load crop calendar data and
  Rank each by crop area in each unit code
  """

  if irrigated_rainfed not in options:
    raise ValueError(f"Invalid value: {value}. Allowed values: {options}")

    
  path_ir = "../data/CropCalender/20{}_{}.csv".format(year, irrigated_rainfed)

  
  if not os.path.isfile(path_ir):
    return(None)

  if not by_state:
    cCropCal_ir = pd.read_csv(path_ir)
    ranked_df_ir = cCropCal_ir.sort_values(['unit_code', 'Crop']) \
                              .groupby('unit_code') \
                              .apply(rank_crops) \
                              .reset_index(level=0, drop=True) \
                              .reset_index() \
                              .sort_values(['unit_code', 'rank'],
                                           ignore_index=True)

  else:
    df_rainfed = pd.read_csv(path_ir)
    # Initialize an empty DataFrame to store the ranked results
    ranked_df_ir = pd.DataFrame()

    # Iterate over unique unit_codes (states) and rank crops for each state
    for unit_code in df_rainfed['unit_code'].unique():
      # print(unit_code)
      state_subset_ir = df_rainfed[df_rainfed['unit_code'] == unit_code]
      ranked_subset_ir = custom_rank(state_subset_ir)
      # ranked_df_ir = ranked_df_ir.append(ranked_subset_ir, ignore_index=True)
      ranked_df_ir = pd.concat([ranked_df_ir, ranked_subset_ir], ignore_index=True)

    # Sort the final DataFrame by 'unit_code' and 'Crop_Rank'
    ranked_df_ir = ranked_df_ir.sort_values(by=['unit_code', 'rank']).reset_index(drop=True)

  return(ranked_df_ir)


def getpath2croptiff(crop_name, root_folder, allcrops, year, unit_code):
  """"
  Get the path to the tiff file
  """
  # GET the TIFF FILE FOR THE CROP
  tiff_file = None
  for k in allcrops.keys():
    if k in crop_name:
      tiff_file = os.path.join("../{}".format(root_folder), 
                         "{}_{}".format(allcrops[k], year), 
                         "{}.tif".format(unit_code))
      # print("FOUND")
      break
  return(tiff_file)

def gdfmask(gdf, raster_path, all_touched=True):
  """ returns a raster mask of the gdf
  
  Parameters
    ----------
    gdf : geopandasdataframe object
        Country
    raster : path to tiff raster
        Administration level

    Returns
    -------
    dictionary of geopandas dataframe's rasters with characteristics of given raster
  
  
  """
  
  with rasterio.open(raster_path) as src:
    # Read the raster data into a NumPy array
    raster_shape = src.read(1).shape
    raster_crs = src.crs
    raster_transform = src.transform

  gdf0 = gdf.to_crs(src.crs)
  gdf_masks = {}
  for index, row in gdf0.iterrows():
    # print(shapely_geometry)
    gdf_masks[index] = geometry_mask([mapping(row['geometry'])],
                                     out_shape=raster_shape,
                                     transform=raster_transform,
                                     all_touched=all_touched, 
                                     invert=True)
  return(gdf_masks)

In [ ]:
# Create Fuctions for parts of the step function

# Functions for parts of MIRCA steps

def calculate_AD_area(step, dist_array, ranked_df_rank, Annual=None):
  """
  Calculate AD_Area
  
  """
  if step in [1]: # uses ranked_df_rf
    # CALCULATE AD
    AD_area = np.where(dist_array["dist_array_ara"] > 0,
                      (AH_array * dist_array["dist_array_aei"]) / dist_array["dist_array_ara"], 0)
    
    # ADJUST AD IF MORE THAN ONE CROP
    if ranked_df_rank.Subcrop.tolist()[0] > 1: 
      AD_area = AD_area/ranked_df_rank.Subcrop.tolist()[0] 

  if step in [5]: # uses ranked_df_rf
    if ranked_df_rank.Crop.tolist()[0] in Annual:
      Pot_ATS5 = dist_array["dist_array_cle"]
    else:
      Pot_ATS5 = np.where(dist_array["dist_array_cle"] > dist_array["dist_array_aei"], 
                          dist_array["dist_array_cle"], 0)
    with np.errstate(divide='ignore'):
      ATS_5 = ranked_df_rank.Growing_area.tolist()[0]
      AD_ra5=np.where(ATS_5<np.sum(Pot_ATS5),(ATS_5/np.sum(Pot_ATS5)), 1)
      AD_area=Pot_ATS5*AD_ra5

  return(AD_area)


def adjust_AD_area(step, dist_array, df_tot, AH_array, AD_area, ATS_2, 
                   listcom, r_s, c_s, ranked_df_rf=None, Annual=None):
  """
  Adjust AD_Area
  
  """
  
  if step in [2, 3, 4, 6, 7]:
    # Filter the total dataframe for a month in which crop grow
    df_max_filt = df_tot[df_tot["Month"].isin(listcom)]
      
    # Group the cell values
    n = len(df_max_filt) // (r_s * c_s)

    # Select the maximum total area assigned for the month in which this crop grow
    df_max = df_max_filt.groupby(np.arange(len(df_max_filt)) // n).max()
    
    # Change it to array
    max_step2 = np.array(df_max["Total_st1"]).reshape(r_s, c_s)

    # Determine the free AEI cell
    AEI_fr_ce_2 = np.where(dist_array["dist_array_aei"] > max_step2, 
                           (dist_array["dist_array_aei"]-max_step2), 0)

  
  if step in [6, 7]:
    CLE_free_6=np.where(dist_array["dist_array_cle"] > max_step2, 
                        (dist_array["dist_array_cle"]-max_step2), 0)
    area_95_=0.95*dist_array["dist_array_ara"]
    free_area_95_6=np.where(area_95_> max_step2, (area_95_-max_step2), 0)

    
  if step in [2]:
  # Determine the AH_free_cell
    AH_fr_ce_2 = np.maximum(AH_array - AD_area, 0)
      
  # Potential area to be distributed in this step
    Pot_ATS2 = np.minimum(AEI_fr_ce_2, AH_fr_ce_2)
        
  if step in [3]:
    Pot_ATS2 = np.where(dist_array["dist_array_cle"] >0, AEI_fr_ce_2, 0)

  if step in [4]:
    Pot_ATS2 = np.where(dist_array["dist_array_cle"] == 0, AEI_fr_ce_2, 0)

      
  if step in [6]: 
    if ranked_df_rf.Crop.tolist()[0] in Annual:
      Pot_ATS2 = np.where(dist_array["dist_array_cle"] > 0,(AEI_fr_ce_2), 0)
    else:
      Pot_ATS2 = np.where(dist_array["dist_array_cle"] > dist_array["dist_array_aei"],
                        (free_area_95_6), 0)

  if step in [7]:
    Pot_ATS2 = np.where(((dist_array["dist_array_cle"] > 0)|(dist_array["dist_array_aei"] > 0)),
                        (free_area_95_6), 0)

    
  # Determine the distribution ratio as the ratio of ATS and Pot_ATS
  with np.errstate(divide='ignore'):
    AD_ra2 = np.where(ATS_2<np.sum(Pot_ATS2),(ATS_2/np.sum(Pot_ATS2)), 1)
    
  # Exact area to be distributed at this step
  if ATS_2>0:
    AD_area2=Pot_ATS2*AD_ra2
    #Area to be distributed in step 1 and 2
                
  else:
    AD_area2= Pot_ATS2*0 if step not in [6, 7] else (Pot_ATS2*AD_ra2)*0

  # TAD_12=AD_area + AD_area2
  AD_area=AD_area + AD_area2

  return(AD_area)


# CREAT FUNCTION TO EASILY SWITCH IRRIGATED AND RAIN TABLES
def distributeArea(step, ranked_df_ir_rank, c_s, r_s, listcom, arr1_re, df_mon):
    # Call the array to be distributed in step1 and convert it to dataframe
    array_name = "Array_" + ranked_df_ir_rank.Crop.tolist()[0][:3] + ranked_df_ir_rank.Crop.tolist()[0][-1] + "1"
    
    # Create a new dataframe in which we will iterate each cell value for each month according to the crop calendar
    #             crop_sum = 0
    df_month = pd.DataFrame(np.arange(1, 13), columns=['Month'])
    df_mon = pd.concat([df_month] * c_s * r_s, ignore_index=True)
    
    # Change the array to dataframe
    df_1 = pd.DataFrame(arr1_re, index=np.arange(len(df_mon)), columns=[array_name])
    # Merge the month and array dataframe
    df_area1 = pd.concat([df_mon, df_1], axis=1)
    # Filter the month in which the crop grows according to the crop calendar
    crop_1 = df_area1[(df_area1["Month"].isin(listcom))]
    # Filter the month in which the crop didn't grow according to the crop calendar
    crop_not = df_area1[(~df_area1["Month"].isin(listcom))]
    # Set the value of the array to 0 for months when the crop doesn't grow
    crop_not[array_name].values[:] = 0
    # Merge the two dataframes and sort it based on index
    df_name = ranked_df_ir_rank.Crop.tolist()[0][:3] + ranked_df_ir_rank.Crop.tolist()[0][-1] + "_step1"
    df_name = pd.concat([crop_1, crop_not])
    df_name = df_name.sort_index()
#             df_name = np.minimum(df_name, dist_array_aei)

    
    # Assign the area to be distributed in this step for each month according to crop calendar
    df_list = []
    df_list.append(df_name[array_name])

    return(df_list, df_mon)

In [ ]:
def step47_tiff_save(step, AD_area, ranked_df_rank, template_tiff, 
                    listcom, df_mon, Step_1, r_s, c_s, year, unit_code, crop_name, 
                    Irrigated= "Irrigated", output_folder = "../scratch"):

  """
  Save step 4 or 7 Tiff
  """
    
  df_irrigaed=pd.concat([df_mon, Step_1], axis=1)
  df_irrigaed.columns = ['Month', 'Total_st1']

  if round(np.sum(AD_area)) < (ranked_df_rank.Growing_area.tolist()[0]):
    print("Error Check Your Code! (full area not assigned)")
    print("Area distributed after step {}: {}".format(step, round(np.sum(AD_area))))
  elif round(np.sum(AD_area))==(ranked_df_rank.Growing_area.tolist()[0]):
    print("Iteration completed ")
    print("Area distributed after step {}: {}".format(step, round(np.sum(AD_area))))
  else:
    # RAINFED CHECK FIX
    if (round(np.sum(AD_area)) - (ranked_df_rank.Growing_area.tolist()[0]))<1:
      print("Iteration completed ")
      print("Area distributed after step {}: {}".format(step, round(np.sum(AD_area))))
        
  if ranked_df_rank.Growing_area.tolist()[0] > 0:
    # Open the existing file to get geotransform and projection
    template_tiff_dataset = gdal.Open(template_tiff)

    # Get geotransform and projection from the input dataset
    geotransform = template_tiff_dataset.GetGeoTransform()
    projection = template_tiff_dataset.GetProjection()
      
    # # Set up the output file
    print(listcom)
    for k in listcom:
      df_max_filt4 = df_irrigaed[(df_irrigaed["Month"] == int(k))]
      arr_filenam = np.array(df_max_filt4["Total_st1"]).reshape(r_s, c_s)
      mask = np.logical_or(np.isnan(arr_filenam), np.logical_or(np.isinf(arr_filenam), arr_filenam == np.NINF))
      arr_filenam = np.where(mask, 0, arr_filenam)
      
        
      # output_tiff_path = "{0}/{1}/District/{2}/{3}/{4}/{5}.tiff".format(output_folder, 
      #                                          Irrigated,
      #                                          year,
      #                                          unit_code,
      #                                          crop_name,
      #                                          calendar.month_name[k]               
      #                                          )
      ir_rf = "ir" if Irrigated == "Irrigated" else "rf"
      output_tiff_path = "{0}/{1}/District/{2}/{3}_{4}_{5}_{6}.tiff".format(output_folder, year, 
                                               Irrigated,
                                               crop_name, unit_code, ir_rf, k               
                                               )
      print(output_tiff_path)
     
      # Create the output directory if it doesn't exist
      output_directory = os.path.dirname(output_tiff_path)
      if not os.path.exists(output_directory):
        os.makedirs(output_directory)

        
      driver = gdal.GetDriverByName("GTiff")
      out_tiff = driver.Create(output_tiff_path, arr_filenam.shape[1], 
                               arr_filenam.shape[0], 1, gdal.GDT_Byte)
      print(out_tiff)
      # Set the geotransform and projection
      out_tiff.SetGeoTransform(geotransform)
      out_tiff.SetProjection(projection)

      # Write the array to the TIFF file
      out_band = out_tiff.GetRasterBand(1)
      out_band.WriteArray(arr_filenam)
      
      # Flush data to disk and close the file
      out_band.FlushCache()
      out_tiff = None
      print("TIFF file saved successfully @ {}".format(output_tiff_path))
  return(True)

In [ ]:
# Create the step function


def step00(step, year, ranked_df_ir_rank, ranked_df_rf_rank, 
           dist_array, AH_array, AD_area,
           c_s=None, r_s=None,
           df_mon=None, df_tot = None, TAD_12=None, df_cumulative=None, 
           ATS_2=None, verbose=False,
           template_tiff= None):
  """

  MIRCA Steps Function

  Function to perform any one of the MIRCA steps
  """

    
  # Get the growing months 
  if step in [1, 2, 3, 4]:
    listcom = growingMonths(ranked_df_ir_rank) 
  elif step in [5, 6, 7]:
    listcom = growingMonths(ranked_df_rf_rank) 

  # If step 1 calculate AD_area
  if step in [1]:
    # CALCULATE AD
    AD_area = calculate_AD_area(step, dist_array, ranked_df_ir_rank)
  # If step 5 recalculates AD_area
  elif step in [5]:
    AD_area = calculate_AD_area(step, dist_array, ranked_df_rf_rank, Annual)

  

  # If step 2, 3, 4, or 6 Adjust AD_area
  if step in [2, 3, 4, 7]:
    AD_area = adjust_AD_area(step, dist_array, df_tot, AH_array, AD_area, 
                             ATS_2, listcom, r_s, c_s, ranked_df_rf_rank)
  # If step 6 Adjust AD_area
  if step in [6]:
    AD_area = adjust_AD_area(step, dist_array, df_tot, AH_array, AD_area, 
                             ATS_2, listcom, r_s, c_s, ranked_df_rf_rank, Annual)


  if step in [1, 2, 3, 5, 6]: 
    if round(np.sum(AD_area),2) <= (ranked_df_ir_rank.Growing_area.tolist()[0]):
      ATS_2= (ranked_df_ir_rank.Growing_area.tolist()[0])-round(np.sum(AD_area),2)
      AD_area=AD_area
    
      if verbose:
        print("Area distributed after step {}:".format(step), round(np.sum(AD_area),2))
        print("Iteration continues")
          
      if round(np.sum(AD_area),2)==(ranked_df_ir_rank.Growing_area.tolist()[0]):
        if step in [5, 6]:
          ATS_2= 0
        if verbose:
          print("Area to be distributed in next step (step {}):".format(step+1), ATS_2)
          print("Iteration completed ")
    else:
      ATS_2= 0
      # ATS_3= 0
      if ranked_df_ir_rank.Growing_area.tolist()[0] >0:
        if verbose:
          print("Error check your code!")
          print("More area available than the growing area")
        ratio= ranked_df_ir_rank.Growing_area.tolist()[0]/np.sum(AD_area)
    #                             print(ratio)
        AD_area=AD_area*ratio
        # AD_area2=TAD_12*ratio
          
        if verbose:
          print("Iteration completed")
          print("Area distributed in step {}:".format(step),round( np.sum(AD_area),2))
      if ranked_df_ir_rank.Growing_area.tolist()[0]== 0:
        AD_area=AD_area*0
        # AD_area2=TAD_12*0
        if verbose:
          print("No area will be distributed in step {} (Zero harvested area)".format(step))


  # return(AD_area)
  if step in [1, 2, 3, 4, 5, 6, 7]: 
    # Call the array (area) to be distributed monthly
    arr1 = AD_area
    arr1_re = np.repeat(arr1, 12)
    c_s = len(arr1[0])
    r_s = len(arr1)
  

  if step in [1, 2, 3, 4]:
    df_list, df_mon = distributeArea(step, ranked_df_ir_rank, c_s, r_s, listcom, arr1_re, df_mon)
  # return(df_list)
  if step in [5, 6, 7]:
    df_list, df_mon = distributeArea(step, ranked_df_rf_rank, c_s, r_s, listcom, arr1_re, df_mon)
    
  Step_1 = pd.concat(df_list, axis=1).sum(axis=1)

  if df_cumulative is None:
    df_cumulative = pd.Series(np.zeros(len(Step_1)), index=Step_1.index)
    
  df_cumulative += Step_1
  df_tot = pd.concat([df_mon, df_cumulative], axis=1)
  df_tot.columns = ['Month', 'Total_st1']
    

  if step in [4]:
    step47_tiff_save(step, AD_area, ranked_df_ir_rank, template_tiff, 
                    listcom, df_mon, Step_1, r_s, c_s, year, unit_code, crop_name, 
                    Irrigated= "Irrigated", output_folder = "../scratch")
  if step in [7]:
    step47_tiff_save(step, AD_area, ranked_df_rf_rank, template_tiff, 
                    listcom, df_mon, Step_1, r_s, c_s, year, unit_code, crop_name, 
                    Irrigated= "Rainfed", output_folder = "../scratch")
      # step47_tiff_save(step, AD_area, ranked_df_rank, template_tiff, 
      #               listcom, df_mon, Step_1, r_s, c_s, year, unit_code, crop_name, 
      #               Irrigated= "Irrigated", output_folder = "../scratch"):


    
  
  return {"df_cumulative":df_cumulative, "AD_area" : AD_area, 
          'df_mon':df_mon, "ATS_2": ATS_2, "df_tot":df_tot,
          "c_s":c_s, "r_s":r_s}

In [ ]:
# Load the datasets


# AEI = '../data/Ras_shape_out/Rasters/AEI/AEI_10.tif'
# Area = '../data/Ras_shape_out/Rasters/Area/ara_10.tif'
# AH = '../data/Ras_shape_out/Rasters/AH'

# # ROOT FOLDER CONTAINS SHOULD CONTAIN CROP SUBFOLDERS WITH CLIPPED AREAS TIFF
# root_folder = {y:"data/Clipped_20{}".format(y) for y in ["00", "05", "10", "15"]}

# # Load the crop and area data clipped by unit code 
# raster_folder = "../data/Ras_shape_out/Rasters"
# tiff_files = glob.glob("{}/**/*.tif".format(raster_folder), recursive=True)
# tiff_files

# Load Crop Calenders
# cCropCal = pd.read_csv("step1/hypothetical_ccc3.csv")
# cCropCal_ir = {}
# cCropCal_rf = {}
# for year in ["00", "05", "10", "15"]:
#   path_ir = "../data/CropCalender/20{}_ir.csv".format(year)
#   if os.path.isfile(path_ir):
#     cCropCal_ir[year] = pd.read_csv(path_ir)
#     # cCropCal_ir[year]
#   path_rf = "../data/CropCalender/20{}_rf.csv".format(year)
#   if os.path.isfile(path_rf):
#     cCropCal_rf[year] = pd.read_csv(path_rf)
#     # cCropCal_rf[year]


# # CLE crop land extent

# # Load the unit code files
# tiff_files = glob.glob("{}/**/*.nc4".format(raster_folder), recursive=True)
# tiff_files

# shp_folder = "../data/Ras_shape_out/Shapefiles"
# shpfiles = glob.glob("{}/**/*.shp".format(shp_folder), recursive=True)
# shpfiles

# MASK data
# # os.listdir(shp_folder)
# # gdf = gpd.read_file(shpfiles[0])
# gdf = gpd.read_file(shpfiles[0]).iloc[:5]
# gdf.plot()

# gdfmask(gdf=gdf, raster_path = AEI)
# gdfmask(gdf=gdf, raster_path = Area)

In [ ]:
# def rasteriosave7(ranked_df, tiff_ir, unit_code, listcom, output_folder):
#   with rasterio.open(tiff_ir) as src:
#     xmin, ymin, xmax, ymax = src.bounds
#     xres, yres = src.res
#     pixel_width, pixel_height = src.res
#     crs = src.crs
      
#     for k in listcom:
#       file_name = ranked_df.Crop[i][:3] + ranked_df.Crop[i][-1] + str(unit_code)+"_tot"
#       df_max_filt4 = df_tot_st4[(df_tot_st4["Month"] == k)]
#       arr_filenam = np.array(df_max_filt4["Total_st4"]).reshape(r_s, c_s)
#       arr_filenam = np.where(np.logical_or(np.isnan(arr_filenam), 
#                                            np.isinf(arr_filenam)), 0, arr_filenam)
#       arr_filenam = arr_filenam.astype(rasterio.float32)  # ensure data type is compatible with output GeoTIFF
#       Ras_filena = ranked_df.Crop[i][:3] + ranked_df.Crop[i][-1] + "_" + str(unit_code) + "_ir" + "_" + str(k)
#       transform = rasterio.transform.from_origin(xmin, ymax, pixel_width, pixel_height)
#       profile = {
#           'driver': 'GTiff',
#           'dtype': arr_filenam.dtype,
#           'count': 1,
#           'height': arr_filenam.shape[0],
#           'width': arr_filenam.shape[1],
#           'transform': transform,
#           'crs': crs,
#           'nodata': 0
#         }
        
#       with rasterio.open(os.path.join(output_folder, Ras_filena + '.tif'), 
#                          'w', **profile) as dst:
#           dst.write(arr_filenam, 1)